In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wheat-plant-diseases


In [28]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import classification_report

In [29]:
BASE_PATH = '/kaggle/input/wheat-plant-diseases/wheat-plant-diseases/'

DATA_DIR_TRAIN = '/kaggle/input/wheat-plant-diseases/data/train'
DATA_DIR_TEST = '/kaggle/input/wheat-plant-diseases/data/test'
IMAGE_SIZE = (224, 224) 
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5

In [30]:
TRAIN_CLASS_FOLDERS = ['Aphid', 'Blast', 'Mildew', 'Smut', 'Tan spot']
TEST_CLASS_FOLDERS = ['aphid_test', 'blast_test', 'mildew_test', 'smut_test', 'tan_spot_test']
REPORT_TARGET_CLASSES = ['Aphid', 'Wheat Blast', 'Powdery Mildew', 'Smut', 'Spot Blotch']

In [31]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

train_datagen = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=resnet_preprocess 
)

test_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess 
)

In [32]:
# TRAINING GENERATOR
print("--- Initializing Data Generators ---")
try:
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='training', shuffle=True
    )
    # VALIDATION GENERATOR
    validation_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='validation', shuffle=False
    )
    # TEST GENERATOR
    test_generator = test_datagen.flow_from_directory(
        DATA_DIR_TEST, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TEST_CLASS_FOLDERS, class_mode='categorical', shuffle=False
    )
    
    print(f"Training Samples: {train_generator.n}")
    print(f"Validation Samples: {validation_generator.n}")
    print(f"Test Samples: {test_generator.n}")

except Exception as e:
    print(f"\nERROR: Could not load data generators. Check the paths and folder names.")
    print(f"Details: {e}")
    exit()

--- Initializing Data Generators ---
Found 3770 images belonging to 5 classes.
Found 941 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Training Samples: 3770
Validation Samples: 941
Test Samples: 250


In [33]:
print("\n--- Building and Training ResNet50 Model ---")

# 1. Load Base Model (ResNet50)
base_model_resnet = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

total_layers = len(base_model_resnet.layers)
UNFREEZE_LAYERS_FROM = total_layers - 60 # 

for layer in base_model_resnet.layers:
    layer.trainable = True

for layer in base_model_resnet.layers[:UNFREEZE_LAYERS_FROM]:
    layer.trainable = False

x = base_model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model_resnet.input, outputs=predictions)


--- Building and Training ResNet50 Model ---


In [34]:
# 3. Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.00001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 19,120,901 (72.94 MB)

 Non-trainable params: 5,518,464 (21.05 MB)

In [35]:
# 4. Train Model
start_time = time.time()
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1
)
training_time = time.time() - start_time

Epoch 1/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 109s 734ms/step - accuracy: 0.3387 - loss: 1.6558 - val_accuracy: 0.6519 - val_loss: 0.9092
Epoch 2/15
  1/117 ━━━━━━━━━━━━━━━━━━━━ 18s 162ms/step - accuracy: 0.5312 - loss: 1.1174

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


117/117 ━━━━━━━━━━━━━━━━━━━━ 15s 127ms/step - accuracy: 0.5312 - loss: 1.1174 - val_accuracy: 0.6573 - val_loss: 0.9064
Epoch 3/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 73s 628ms/step - accuracy: 0.6558 - loss: 0.9024 - val_accuracy: 0.7317 - val_loss: 0.7115
Epoch 4/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 15s 127ms/step - accuracy: 0.8750 - loss: 0.5354 - val_accuracy: 0.7306 - val_loss: 0.7134
Epoch 5/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 73s 625ms/step - accuracy: 0.7384 - loss: 0.6893 - val_accuracy: 0.7683 - val_loss: 0.6080
Epoch 6/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 15s 125ms/step - accuracy: 0.7500 - loss: 0.6854 - val_accuracy: 0.7619 - val_loss: 0.6103
Epoch 7/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 73s 621ms/step - accuracy: 0.8004 - loss: 0.5340 - val_accuracy: 0.7942 - val_loss: 0.5442
Epoch 8/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 15s 127ms/step - accuracy: 0.8438 - loss: 0.4186 - val_accuracy: 0.7942 - val_loss: 0.5501
Epoch 9/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 73s 622ms/step - accuracy: 0.8279 - loss: 0.4715 - val

In [36]:
# 5. Evaluate and Report
print(f"\n--- FINAL RESULTS ---")

# Evaluate on Test Generator
loss, accuracy = model.evaluate(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


--- VGG19 FINAL RESULTS ---
Test Accuracy: 0.9040
Training Time: 724.94 seconds


In [37]:
# Generate Classification Report
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

report = classification_report(
    y_true, 
    y_pred, 
    target_names=REPORT_TARGET_CLASSES, 
    zero_division=0,
    output_dict=True 
)

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=REPORT_TARGET_CLASSES, zero_division=0))


--- CLASSIFICATION REPORT ---
                precision    recall  f1-score   support

         Aphid       0.86      0.88      0.87        50
   Wheat Blast       0.94      1.00      0.97        50
Powdery Mildew       0.96      0.86      0.91        50
          Smut       0.94      1.00      0.97        50
   Spot Blotch       0.81      0.78      0.80        50

      accuracy                           0.90       250
     macro avg       0.90      0.90      0.90       250
  weighted avg       0.90      0.90      0.90       250



In [38]:
model_name = "ResNet50"

metrics_summary = {
    'Model': model_name,
    'Accuracy': report['accuracy'],
    'Precision': report['weighted avg']['precision'], 
    'Recall': report['weighted avg']['recall'],
    'F1-Score': report['weighted avg']['f1-score'],
    'Training Time (s)': training_time,
}

df_metrics = pd.DataFrame([metrics_summary])


In [39]:
results_file = 'model_performance_summary.csv'
if not os.path.exists(results_file):
    df_metrics.to_csv(results_file, index=False)
else:
    df_metrics.to_csv(results_file, mode='a', header=False, index=False)
    
print(f"\nMetrics saved to {results_file}")


Metrics saved to model_performance_summary.csv
